In [1]:
import quanguru as qg
import numpy as np
import matplotlib.pyplot as plt

# Simulation of the dynamics of a Qubit

In previous tutorials, the system Hamiltonians were time independent, and QuanGuru provides couple different methods to describe and simulate time-dependent Hamiltonians.
In this tutorial, we demonstrate one (probably the most convenient and flexible) of these time-dependent Hamiltonian methods.

Here, we use the below single qubit Hamiltonian,
\begin{equation}
H=f_{z}J_{z} + f_{x}(t)\sigma_{x}
\end{equation}
where the second term will be time dependent.

We des

In [2]:
qubFreq = 1
driveFreq = 2
driveAmp = 1
drivePhase = 0
couplingStrength = 1

OmegaR = couplingStrength*driveAmp
detun = qubFreq-driveFreq
Omega = np.sqrt((OmegaR**2) + (detun**2))

In [3]:
qub = qg.QuantumSystem(operator=qg.Jz)
qub.dimension = 2
qub.frequency = qubFreq

# add the drive terms 
secondTerm = qub.addTerm(operator=qg.sigmax)
secondTerm.frequency = driveFreq

qub.initialState = 1

qub.simTotalTime = 2*np.pi
qub.simStepSize = 0.01

# create the operators for which we compute the expectation values
sigmaX = qg.sigmax()

# write a compute function that takes two arguments: (i) a quantum-system (qsys) and (ii) a state
# compute whatever we want and store in .results
def compute(qsys, state):
    qsys.results['sigmax expectation'].append(qg.expectation(sigmaX, state))

# set the compute attribute of our qubit to compute function
qub.compute = compute

In [4]:
def secondTermTime(st, ti):
    return couplingStrength*driveAmp*np.cos(2*np.pi*driveFreq*ti + drivePhase)

secondTerm.timeDependency = secondTermTime

In [5]:
states = qub.runSimulation()

In [6]:
qub._freeEvol.numberOfExponentiations, len(qub.simulation.timeList)

(629, 630)

In [7]:
plt.plot(qub.simulation.timeList, qub.results['sigmax expectation'])